This file is going to explain how to use each module, in what order, and what they each do.

First is the "card_handler.py" module. This, as the name implies, handles requests to cards.
"deck_handler.py", as you probably guessed, handles every action to do with the deck.
"deck_simulator" contains functions that can, in simple terms, see if the mana ratios are reasonable.

So, to get started you should first find a "commander". A commander must either be a 'Legendary Creature', or a 'Planeswalker' with an effect that allows it to be a commander. TO make it easy, use the search text "is:commander legal:commander" on scryfall.com. I have chosen not to implement this here as scryfall has a better layout.

The reason you need a commander first is because the "color identity" of your commander determines which color cards you are allowed to play.

For the purpose of this exam, some examples to use could be "Atraxa, Praetor's Voice", "Karametra, God of Harvests", or "Master Transmuter".

In order to begin, use the "deck_handler.begin_deck_construction" function. This takes two arguments, a deck name and a card name.
It will return a string if it doesn't find a valid card, or too many.

In [1]:
from modules import deck_handler

#lets try with "Karametra, God of Harvests". The deck shall be called "Karamatra_advanced"
#in this case, because the name contains multiple words we need to surround it in quotation marks
deck_handler.begin_deck_construction("Karametra_advanced", '"karametra, god"')
#this makes a .csv file in the "decks" folder with the chosen name

['Karametra, God of Harvests', 'Legendary Enchantment Creature — God', "['3', 'G', 'W']", 5.0, "['G', 'W']"]


'Success!'

In [2]:
#next, how about we find some more cards?
from modules import card_handler

#Using webscraping and mtggoldfish, we can get data on what cards other people play and what percentages of decks are using them
deck = 'Karametra_advanced'

#list 0 is creatures, 1 is planeswalkers, 2 is spells, 3 is artifacts, 5 is enchantments, 6 is lands
#let's use creatures as an example
for c in card_handler.get_archetype(deck)[0][0:10]: #just doing 10 for speed reasons
    print(c[0] + ": " + str(card_handler.get_effect_text(c[0])))

Beast Whisperer: ['Whenever you cast a creature spell, draw a card.']
Tireless Provisioner: ['Landfall — Whenever a land enters the battlefield under your control, create a Food token or a Treasure token. (Food is an artifact with "{2}, {T}, Sacrifice this artifact: You gain 3 life." Treasure is an artifact with "{T}, Sacrifice this artifact: Add one mana of any color.")']
Rampaging Baloths: ['Trample', 'Landfall — Whenever a land enters the battlefield under your control, you may create a 4/4 green Beast creature token.']
Emeria Shepherd: ['Flying', 'Landfall — Whenever a land enters the battlefield under your control, you may return target nonland permanent card from your graveyard to your hand. If that land is a Plains, you may return that nonland permanent card to the battlefield instead.']
Avacyn's Pilgrim: ['{T}: Add {W}.']
Sakura-Tribe Elder: ['Sacrifice Sakura-Tribe Elder: Search your library for a basic land card, put that card onto the battlefield tapped, then shuffle.']
Scut

Now lets use that "Hypergeometric formula" I mentioned earlier, shall we?

The hypergeometric formula is a simple formula to calculate the chance of getting 'x' successes in 'n' samples with a population of N containing k total successes.

So how do we apply this to our problem of finding how much mana and ramp spells to play?
Well first we must define what it is we are ramping into. "Atraxa, Praetors' Voice" costs 4 total mana, so lets use that.

Now lets define our values:
N is population size, in this case our deck size which will always be 99
k is the number of successes in our population, in this case the number of ramp spells
n is the sample size. Due to the rules of commander this number is 7+the number of turns. We want to get our commander at least one turn early, so n will be 10.
x is the number of successes in the samples, here 1 because we want at least 1 ramp spell before turn 4.

In [10]:
from modules import deck_simulator

N = 99
k = 14
n = 10
x = 1

print(("The chance of %r ramp before turn %r: "+str(deck_simulator.hyper_calc_fin(N, k, n, x))) % (x, (n-7))) #this calculates for x or GREATER in n

N = 99
k = 36
n = 10
x = 3
print(("The chance of %r lands before turn %r: "+str(deck_simulator.hyper_calc_fin(N, k, n, x))) % (x, (n-7)))

The chance of 1 ramp before turn 3: 0.7991458544457899
The chance of 3 lands before turn 3: 0.7804763422171944


We could play more lands and ramp to increase these odds but that would also increase our chances of drawing bad cards. After all, what use is all this mana if you have nothing to spend it on.

Deciding what ramp to play can be tough Luckily, we have others to help us out. By once again using both mtggoldfish and scryfall, we can get a list of ramp cards other players are using in a similar deck. Due to the number and variety of different decks, even using the same commander, the number is going to be much higher than the number of ramp we want to play and thus we need to pick the cards that would work best for OUR deck.

In [4]:
from modules import deck_handler, card_handler

#once again, only doing 10 for the sake of speed
for c in deck_handler.get_meta_ramp('atraxa_advanced')[0:10]:
    print(c+": "+str(card_handler.get_effect_text(c)))

Birds of Paradise: ['Flying', '{T}: Add one mana of any color.']
Faeburrow Elder: ['Vigilance', 'Faeburrow Elder gets +1/+1 for each color among permanents you control.', '{T}: For each color among permanents you control, add one mana of that color.']
Crystalline Crawler: ['Converge — Crystalline Crawler enters the battlefield with a +1/+1 counter on it for each color of mana spent to cast it.', 'Remove a +1/+1 counter from Crystalline Crawler: Add one mana of any color.', '{T}: Put a +1/+1 counter on Crystalline Crawler.']
Bloom Tender: ['{T}: For each color among permanents you control, add one mana of that color.']
Plague Myr: ['{TK}{TK} — Whenever this creature attacks, scry 1.', '{TK}{TK}{TK} — Whenever this permanent leaves the battlefield, you may destroy target artifact enchantment.', '{TK}{TK}{TK} — 4/5', '{TK}{TK}{TK}{TK} — 8/4']
Biophagus: ['Genomic Enhancement — {T}: Add one mana of any color. If this mana is spent to cast a creature spell, that creature enters the battlefi